In [23]:
import pandas as pd
import numpy as np
import hopsworks
import os 
# from dotenv import load_dotenv
import requests
import json
import datetime
import urllib.request
import codecs
import csv
import sys
from functions import *

import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import f1_score
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

In [2]:
import hopsworks

project = hopsworks.login() 

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5359
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
feature_view = fs.get_feature_view(
    name = 'air_quality_fv',
    version = 5
)

In [8]:
# feature_view.create_training_data()

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/5359/jobs/named/air_quality_fv_5_1_create_fv_td_13012023162707/executions


(1, <hsfs.core.job.Job at 0x7fc2e5cc53d0>)

In [16]:
training_data = feature_view.get_training_data(1)[0]
training_data.head()

,aqi,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,date,o3_avg,o3_max,o3_min,pm10_avg,...,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,21,1.576445,-2.698954,-1.403590,0.484899,1673136000000,2.380214,1.010450,2.409674,-2.243365,...,2.671626,2.292059,0.528172,-2.458633,1.427989,-0.339022,-2.299722,-2.299049,-1.784827,2
1,46,-1.339579,-0.455988,2.299942,0.734550,1672725600000,0.068397,-2.151883,0.651596,-0.901101,...,-0.091507,-0.445865,-1.360374,0.737590,-0.481071,-1.176771,0.559105,0.572582,0.615457,0
2,53,-1.910912,0.425178,0.041691,2.107632,1672639200000,0.424061,0.767193,-0.147531,-0.413004,...,-0.613922,0.194926,0.217430,-0.577169,0.916471,-1.176771,-0.706384,-0.674310,-0.984732,1
3,78,-0.055935,0.104754,0.854661,-0.295260,1672898400000,-0.465099,0.523937,-0.786832,0.319140,...,-0.639099,-0.640044,-1.321324,0.669585,-1.546735,-0.739684,0.723029,0.723721,0.615457,3
4,53,0.329901,1.092727,0.132021,-0.388880,1673308800000,0.068397,0.037424,-0.307356,0.197116,...,-0.299214,-0.562372,0.053750,0.624249,-0.706383,1.409323,0.726307,0.685936,0.615457,0


In [17]:
training_data = training_data.sort_values(by=["date"], ascending=[False]).reset_index(drop=True)
training_data["aqi_next_day"] = training_data['aqi'].shift(1)

training_data

,aqi,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,date,o3_avg,o3_max,o3_min,pm10_avg,...,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions,aqi_next_day
0,78,0.144403,0.959217,0.674001,-1.325071,1673568000000,-0.998596,-0.205832,-0.786832,0.807236,...,-0.523537,-1.322154,0.442903,0.557190,0.389455,0.713194,0.723721,0.615457,0,None
1,38,-0.204333,1.333045,-0.500290,-1.231452,1673481600000,-0.642931,0.767193,-0.307356,-0.168956,...,-0.368193,1.364017,0.828263,0.876889,1.008661,0.739421,0.723721,0.615457,0,78
2,25,1.205450,0.612092,-0.951940,-0.295260,1673395200000,-0.820763,-0.449089,-0.627007,0.563188,...,-0.309940,1.053275,0.533576,0.843397,0.535150,-0.368702,-0.409818,-0.984732,1,38
3,53,0.329901,1.092727,0.132021,-0.388880,1673308800000,0.068397,0.037424,-0.307356,0.197116,...,-0.562372,0.053750,0.624249,-0.706383,1.409323,0.726307,0.685936,0.615457,0,25
4,21,-0.723727,-0.909921,-1.403590,1.171440,1673222400000,1.668886,1.740219,0.651596,-1.145149,...,2.301768,0.790723,-0.894524,-0.429310,1.409323,0.559105,0.610367,-0.184637,1,53
5,21,1.576445,-2.698954,-1.403590,0.484899,1673136000000,2.380214,1.010450,2.409674,-2.243365,...,2.292059,0.528172,-2.458633,1.427989,-0.339022,-2.299722,-2.299049,-1.784827,2,21
6,85,1.576445,-0.402584,0.854661,-1.449897,1673049600000,0.246229,0.280680,-0.946658,-0.413004,...,-0.533246,-0.170582,-0.667842,1.132648,-0.193327,-1.883356,-1.883419,-1.784827,1,21
7,34,-0.041095,0.104754,-0.409960,0.422487,1672963200000,-0.998596,-1.422114,-0.786832,1.539380,...,-0.601208,-0.103282,-0.078467,-1.242259,1.190780,0.834497,0.837075,0.615457,3,85
8,78,-0.055935,0.104754,0.854661,-0.295260,1672898400000,-0.465099,0.523937,-0.786832,0.319140,...,-0.640044,-1.321324,0.669585,-1.546735,-0.739684,0.723029,0.723721,0.615457,3,34
9,65,-0.649528,0.051350,0.403011,-0.482499,1672812000000,-0.998596,-0.692345,-0.627007,1.295332,...,-0.474992,1.486984,1.530979,-1.476705,-1.176771,0.100119,0.081382,0.615457,3,78


In [18]:
training_data = training_data.iloc[1:]

In [19]:
X = training_data.drop(columns=["date"]).fillna(0)
y = training_data["aqi_next_day"]

In [24]:
model = xgb.XGBRegressor(n_estimators=10000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

In [25]:
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False, eta=0.1,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.100000001, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=7, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=10000, n_jobs=0,
             num_parallel_tree=1, predictor='auto', ...)

In [27]:
y_pred = model.predict(X)
y_pred = np.array([int(x) for x in y_pred])

In [29]:
f1score = f1_score(y.astype(int), y_pred, average="micro")

In [31]:
f1score

0.5833333333333334

In [33]:
pred_comparison_df = pd.DataFrame({
    "aqi_true": y.astype(int),
    "aqi_pred": y_pred

})

pred_comparison_df

,aqi_true,aqi_pred
1,78,77
2,38,38
3,25,25
4,53,53
5,21,20
6,21,21
7,85,84
8,34,34
9,78,77
10,65,65


# Model Registry

In [34]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [35]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X)
output_schema = Schema(y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'aqi', 'type': 'int64'},
   {'name': 'iaqi_h', 'type': 'float64'},
   {'name': 'iaqi_p', 'type': 'float64'},
   {'name': 'iaqi_pm10', 'type': 'float64'},
   {'name': 'iaqi_t', 'type': 'float64'},
   {'name': 'o3_avg', 'type': 'float64'},
   {'name': 'o3_max', 'type': 'float64'},
   {'name': 'o3_min', 'type': 'float64'},
   {'name': 'pm10_avg', 'type': 'float64'},
   {'name': 'pm10_max', 'type': 'float64'},
   {'name': 'pm10_min', 'type': 'float64'},
   {'name': 'pm25_avg', 'type': 'float64'},
   {'name': 'pm25_max', 'type': 'float64'},
   {'name': 'pm25_min', 'type': 'float64'},
   {'name': 'city', 'type': 'int64'},
   {'name': 'tempmax', 'type': 'float64'},
   {'name': 'tempmin', 'type': 'float64'},
   {'name': 'temp', 'type': 'float64'},
   {'name': 'feelslikemax', 'type': 'float64'},
   {'name': 'feelslikemin', 'type': 'float64'},
   {'name': 'feelslike', 'type': 'float64'},
   {'name': 'dew', 'type': 'float64'},
   {'name': 'humidity',

In [36]:
import joblib

joblib.dump(model, "model.pkl")

['model.pkl']

In [39]:
model_reg = mr.python.create_model(
    name="xgboost_model",
    metrics={"f1": f1score},
    description="XGB Regressor for Air quality",
    input_example=X.sample().to_numpy(),
    model_schema=model_schema
)

model_reg.save('model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/5359/models/xgboost_model/2


Model(name: 'xgboost_model', version: 2)